# Objects, Properties and Rules

### Objects
In polymake, there is two kinds of objects. A *Big Object* models a complex mathematical concept, like a Polytope or a SimplicialComplex, while a *small object* is an instance of one of the many data types commonly used in computer science, like Integers, Matrices, Sets or Maps. A big object consists of a collection of other objects (big or small) describing it, called *properties*, and functions to compute more properties from the ones already known, called *production rules*.

To get a more detailed explanation of the `polymake` object model and properties, check out the [scripting guide](https://polymake.org/doku.php/user_guide/howto/scripting#most_important_interfaces).

You can save polymake objects to disc, as explained [here](data.ipynb).

### Properties

Each (big) object has a list of properties of various types.  When an object is 'born' it comes with an initial list of properties, and all other properties will be derived from those.  Let's look at example from the `polytope` application.  The following creates a 3-dimensional cube:

In [1]:
$c=cube(3);

To find out what the initial set of properties is, use the `list_properties` method.  It returns an array of strings.  The extra code is just there to print this list nicely.

In [2]:
print join(", ", $c->list_properties);

CONE_AMBIENT_DIM, CONE_DIM, FACETS, AFFINE_HULL, VERTICES_IN_FACETS, BOUNDED

To find out the type of the object `$c`, enter

In [3]:
print $c->type->full_name;

Polytope<Rational>


To see what a property contains, use the `->` syntax:

    


In [4]:
print $c->FACETS;

1 1 0 0
1 -1 0 0
1 0 1 0
1 0 -1 0
1 0 0 1
1 0 0 -1


    
    1 1 0 0
    1 -1 0 0
    1 0 1 0
    1 0 -1 0
    1 0 0 1
    1 0 0 -1


You can also get the content of all properties using the `properties` method:

    


In [5]:
$c->properties;

name: c
type: Polytope<Rational>
description: cube of dimension 3


AFFINE_HULL


BOUNDED
true

CONE_AMBIENT_DIM
4

CONE_DIM
4

FACETS
1 1 0 0
1 -1 0 0
1 0 1 0
1 0 -1 0
1 0 0 1
1 0 0 -1


VERTICES_IN_FACETS
{0 2 4 6}
{1 3 5 7}
{0 1 4 5}
{2 3 6 7}
{0 1 2 3}
{4 5 6 7}



### Production Rules

The object is changed if we ask for a property which has not been computed before.

In [6]:
print $c->VERTICES;

1 -1 -1 -1
1 1 -1 -1
1 -1 1 -1
1 1 1 -1
1 -1 -1 1
1 1 -1 1
1 -1 1 1
1 1 1 1


In [7]:
print join(", ", $c->list_properties);

CONE_AMBIENT_DIM, CONE_DIM, FACETS, AFFINE_HULL, VERTICES_IN_FACETS, BOUNDED, FEASIBLE, POINTED, N_VERTICES, N_FACETS, VERTICES, LINEALITY_SPACE


The property `VERTICES` was added, but a few others were computed on the way, too. `polymake` applied a sequence of *production rules* that add new properties to the object that can be computed from the properties the object already posesses.

What properties *can* be computed for a given object depends on the set of rules defined for it. Here is a short sequence of commands which lets you find out.

    


In [8]:
$t=$c->type;
print join(", ", sorted_uniq(sort { $a cmp $b } map { keys %{$_->properties} } $t, @{$t->super}));

AFFINE_HULL, BALANCE, BALANCED, BOUNDARY_LATTICE_POINTS, BOUNDED, CANONICAL, CD_INDEX_COEFFICIENTS, CENTERED, CENTERED_ZONOTOPE, CENTRALLY_SYMMETRIC, CENTROID, CHIROTOPE, CIRCUITS, COCIRCUITS, COCIRCUIT_EQUATIONS, COCUBICAL, COCUBICALITY, COMBINATORIAL_DIM, COMPLEXITY, COMPRESSED, CONE_AMBIENT_DIM, CONE_DIM, CS_PERMUTATION, CUBICAL, CUBICALITY, CUBICAL_H_VECTOR, DEGREE_ONE_GENERATORS, DUAL_BOUNDED_H_VECTOR, DUAL_GRAPH, DUAL_H_VECTOR, EDGE_ORIENTABLE, EDGE_ORIENTATION, EHRHART_POLYNOMIAL, EHRHART_QUASI_POLYNOMIAL, EQUATIONS, EXCESS_RAY_DEGREE, EXCESS_VERTEX_DEGREE, F2_VECTOR, FACETS, FACETS_THRU_INPUT_RAYS, FACETS_THRU_POINTS, FACETS_THRU_RAYS, FACETS_THRU_VERTICES, FACET_SIZES, FACET_VERTEX_LATTICE_DISTANCES, FACET_VOLUMES, FACET_WIDTH, FACET_WIDTHS, FACE_SIMPLICITY, FAR_FACE, FAR_HYPERPLANE, FATNESS, FEASIBLE, FLAG_VECTOR, FOLDABLE_COCIRCUIT_EQUATIONS, FOLDABLE_MAX_SIGNATURE_UPPER_BOUND, FTR_CYCLIC_NORMAL, FTV_CYCLIC_NORMAL, FULL_DIM, F_VECTOR, FacetPerm, FacetPerm.pure, GALE_TRANSFOR



Instead of showing the (lengthy) enumeration have a look at the [documentation](https://polymake.org/doku.php/documentation/latest/polytope) for a complete list of properties known for objects of the application `polytope`.


#### Schedules

You may wonder what sequence of rules led to the computation of a property you request. There usually are several mathematical ways to compute a property. `polymake` uses a nice scheduling algorithm to find the most efficient procedure, and you can look at what it returns.

Suppose we want to see which sequence of rules leads to the computation of the F_VECTOR.

    


In [9]:
$schedule=$c->get_schedule("F_VECTOR");
print join("\n", $schedule->list);

LINEALITY_DIM : LINEALITY_SPACE
COMBINATORIAL_DIM : CONE_DIM, LINEALITY_DIM
precondition : COMBINATORIAL_DIM ( F_VECTOR : N_FACETS, N_RAYS, COMBINATORIAL_DIM )
F_VECTOR : N_FACETS, N_RAYS, COMBINATORIAL_DIM

So if you ask for the f-vector, `polymake` will first compute the dimension of the lineality space from the basis of the lineality space, then compute the combinatorial dimension from the lineality and cone dimensions, and then compute the f-vector from the number of facets, number of rays, and combinatorial dimension of the polytope. Applying the schedule to the object yields the same as asking for the property right away:

In [10]:
$schedule->apply($c);
print join(", ", $c->list_properties);

CONE_AMBIENT_DIM, CONE_DIM, FACETS, AFFINE_HULL, VERTICES_IN_FACETS, BOUNDED, FEASIBLE, POINTED, N_VERTICES, N_FACETS, VERTICES, LINEALITY_SPACE, LINEALITY_DIM, COMBINATORIAL_DIM, F_VECTOR

As you can see, the things `polymake` needed to compute in order to get to the f-vector are stored in the object as well, so you don't have to recompute them later.

If you're interested, read more about rule scheduling in the [scripting guide](https://polymake.org/doku.php/user_guide/howto/scripting#rule_planning) and the article on [writing rules yourself](https://polymake.org/doku.php/user_guide/extend/rulefiles).